
# Spot resources Analytics

Here we perform some initial process and analysis on the dataset.

---




In [1]:
import pandas as pd

# parse the data file and extra the results
filename = 'aws-spot-price-history/data-1397804701'

df = pd.read_csv(filename, sep="\t", header = None)
df.columns = ["info", "Spot price", "Time", "Machine size", "OS type", "Region"]
df = df.drop('info', 1)

df.head(10)

,Spot price,Time,Machine size,OS type,Region
0,0.0812,2014-04-17T17:35:33-0700,c3.2xlarge,Linux/UNIX,us-west-1a
1,0.0699,2014-04-17T17:31:34-0700,c3.2xlarge,Linux/UNIX,us-west-1a
2,0.1016,2014-04-17T17:29:35-0700,c3.2xlarge,Linux/UNIX,us-west-1b
3,0.0709,2014-04-17T17:29:35-0700,c3.2xlarge,Linux/UNIX,us-west-1a
4,0.0984,2014-04-17T17:27:35-0700,c3.2xlarge,Linux/UNIX,us-west-1b
5,0.0699,2014-04-17T17:23:36-0700,c3.2xlarge,Linux/UNIX,us-west-1a
6,0.0692,2014-04-17T17:17:38-0700,c3.2xlarge,Linux/UNIX,us-west-1a
7,0.0686,2014-04-17T17:15:38-0700,c3.2xlarge,Linux/UNIX,us-west-1a
8,0.0677,2014-04-17T17:13:38-0700,c3.2xlarge,Linux/UNIX,us-west-1a
9,0.0681,2014-04-17T17:11:38-0700,c3.2xlarge,Linux/UNIX,us-west-1a


## Hypothesis #1
**Problems:** Can we predict future price of a spot instance given previous history and how other vm’s are reacting?

To achieve the goal of prediction, we are expecting to do pattern matching from the collected dataset. In this case, whenever users make a bid, we can based on the resources types, time or day, and the trending price to do pattern matching. We will be able to provide a prediction if we can shoot a pattern.

Expecting tools are supervised and unsupervised learning algorithms, e.g. classification and
clustering methods.



## Hypothesis #2

For each machine type there exists a region that is more favorable to use, as the market volatility is very low and the prices tend to stay cheaper than the other regions.

With in proving this hypothesis users will be able to find the best region they should be bidding in, as long as latency is not an issue for them.

Data Science tools & Techniques: We can use clustering and classification methods.


## Hypothesis #3

There exists some kind of relation between what kind of virtual machines are turning into hotspots. Say that we establish a line as half price of EC2 instances, it makes sense to pay half price to gain usage of resources but probably not more than 3⁄4. By extracting patterns from the price history, we can study that whether or not there was the case that some resources were becoming hotspot in the spot instances market.

Potential data science method for this one includes: Time Series, Linear Regression
